In [20]:
import pandas as pd
import asyncio, json
from aiohttp import ClientSession
from selectolax.parser import HTMLParser


df = pd.read_excel("prices3.xlsx")
df.columns = [c.lower().strip() for c in df.columns]
df["sku"] = df["sku"].astype(str)


API_URL  = "https://ksa-api.boutiqaat.com/searchplus/rest/V2/global/suggest"
PAGE_URL = "https://www.boutiqaat.com/en-kw/men/{slug}"

HEADERS = {
    "accept": "application/json, text/plain, */*",
    "content-type": "application/json",
    "origin": "https://www.boutiqaat.com",
    "referer": "https://www.boutiqaat.com/",
    "user-agent": "Mozilla/5.0",
    "param": '{"gender":"4194","app_version":"8.0.0","platform":"web","device_type":"desktop"}'
}

BASE_PAYLOAD = {
    "countryCodeAndLanguage": "kw_en",
    "numberOfRecords": 20,
    "featuredOnly": None,
    "searchString": ""
}

async def fetch_slug(session, sku):
    payload = dict(BASE_PAYLOAD)
    payload["searchString"] = sku
    async with session.post(API_URL, headers=HEADERS, json=payload) as r:
        j = await r.json(content_type=None)
    try:
        return j["data"][0]["data"][0]["slug"]
    except:
        return None

async def fetch_price(session, slug):
    url = PAGE_URL.format(slug=slug)
    async with session.get(url, headers=HEADERS) as r:
        html = await r.text()
    tree = HTMLParser(html)
    script = tree.css_first('script#__NEXT_DATA__')
    j = json.loads(script.text())
    try:
        return float(j["props"]["pageProps"]["response"][0]["final_price_with_tax"])
    except:
        return None

async def process_row(session, sku, expected_price):
    slug = await fetch_slug(session, sku)
    web_price = await fetch_price(session, slug) if slug else None
    match = (web_price == expected_price) if web_price is not None else None
    return {
        "sku": sku,
        "excel_price": expected_price,
        "web_price": web_price,
        "match": match
    }

# Run
async def main(df):
    async with ClientSession() as session:
        tasks = [
            process_row(session, row["sku"], float(row["special price"]))
            for _, row in df.iterrows()
        ]
        return await asyncio.gather(*tasks)

out = await main(df)
results = pd.DataFrame(out)

# Save
results.to_excel("comparison_results.xlsx", index=False)
results.to_csv("comparison_results.csv", index=False)

results.head(10)


,sku,excel_price,web_price,match
0,ORL-00006341,6.600,6.60,True
1,ORL-00006341,6.601,6.60,False
2,I-00000190608,15.000,15.00,True
3,I-00000190608,15.001,15.00,False
4,ORL-00006239,22.500,18.00,False
5,ORL-00006239,22.501,18.00,False
6,ORL-00004724,17.130,17.13,True
7,ORL-00004724,17.131,17.13,False
8,MU-00022119,8.500,8.50,True
9,MU-00022119,8.501,8.50,False
